In [1]:
from __future__ import print_function, division
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras
from keras import backend as K
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
from skimage.transform import resize
import skimage.io
import scipy
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import sys
from tensorflow.python.client import device_lib 
!pip install git+https://www.github.com/keras-team/keras-contrib.git
%matplotlib inline

print('Tensorflow Version: ', tf.__version__)
print('Tensorflow built with Cuda: ', tf.test.is_built_with_cuda())
print('Tensorflow built with GPU support: ', tf.test.is_built_with_gpu_support())
print('GPU available: ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None), '\n')
print(device_lib.list_local_devices())

Using TensorFlow backend.


  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-owliqmnp
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-owliqmnp
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=100247 sha256=35f7fa40d7710d8cec29e5bf89e9508ab7692aee912c177e22d697dc17f7819c
  Stored in directory: /tmp/pip-ephem-wheel-cache-sr8bz9rw/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Tensorflow Version:  1.15.0
Tensorflow built with Cuda:  True
Tensorflow built with GPU support:  True
GPU available:  True 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4208549595547556548
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17871391701678559805
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limi

In [2]:
# Set directory
directory = 'SRGAN/non_demented'
# Append list of image paths
filepaths = []

for dir_, _, files in os.walk(directory):
    for fileName in files:
        relDir = os.path.relpath(dir_, directory)
        relFile = os.path.join(relDir, fileName)
        filepaths.append(directory + "/" + relFile)

In [4]:
# Load Data Function
def load_data(batch_size=1, is_testing=False):
    """
    Establish random image selection for training.
    Resizes images and scales images -1,1
    Outputs high-resolution image, low-resolution image
    """
    data_type = "train" if not is_testing else "test"

    batch_images = np.random.choice(filepaths, size=1)

    imgs_hr = []
    imgs_lr = []
    for img_path in batch_images:
        img = cv2.imread(img_path).astype(np.float)
        h, w = (720, 720) # 256?
        low_h, low_w = int(h / 4), int(w / 4)
        img_hr = resize(img, (720, 720, 3))
        img_lr = resize(img, (low_h, low_w, 3))

        if not is_testing and np.random.random() < 0.5:
            img_hr = np.fliplr(img_hr)
            img_lr = np.fliplr(img_lr)

        imgs_hr.append(img_hr)
        imgs_lr.append(img_lr)

    imgs_hr = np.array(imgs_hr) / 719.5 - 1.
    imgs_lr = np.array(imgs_lr) / 719.5 - 1.

    return imgs_hr, imgs_lr

In [5]:
# Network Attributes
dataset_name = 'non_demented'
channels = 3                  # Channel
lr_height = 180                # Low resolution height
lr_width = 180                 # Low resolution width
lr_shape = (lr_height, lr_width, channels)          # Low resolution shape
hr_height = lr_height*4   # High resolution height
hr_width = lr_width*4     # High resolution width
hr_shape = (hr_height, hr_width, channels)          # High resolution shape
n_residual_blocks = 16    # Number of residual blocks in the generator
optimizer = Adam(0.0002, 0.5)     # Optimizer

In [6]:
# Construct VGG19 Pre-Trained Model for Feature Extraction
def build_vgg(hr_shape=hr_shape):
    """
    Builds a pre-trained VGG19 model that outputs image features extracted at the
    third block of the model
    """
    vgg = VGG19(weights="imagenet")
    # Set outputs to outputs of last conv. layer in block 3
    # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
    vgg.outputs = [vgg.layers[9].output]

    img = Input(shape=hr_shape)

    # Extract image features
    img_features = vgg(img)

    return Model(img, img_features)

# We use a pre-trained VGG19 model to extract image features from the high resolution
# and the generated high resolution images and minimize the mse between them
vgg = build_vgg(hr_shape)
vgg.trainable = False
vgg.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

In [7]:
# Configure data loader
# Calculate output shape of D (PatchGAN)
patch = int(hr_height / 2**4)
disc_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
gf = 64
df = 64

def build_discriminator(hr_shape=hr_shape, df=df):

    def d_block(layer_input, filters, strides=1, bn=True):
        """
        Discriminator layer
        """
        d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if bn:
            d = BatchNormalization(momentum=0.8)(d)
        return d

    # Input img
    d0 = Input(shape=hr_shape)

    d1 = d_block(d0, df, bn=False)
    d2 = d_block(d1, df, strides=2)
    d3 = d_block(d2, df*2)
    d4 = d_block(d3, df*2, strides=2)
    d5 = d_block(d4, df*4)
    d6 = d_block(d5, df*4, strides=2)
    d7 = d_block(d6, df*8)
    d8 = d_block(d7, df*8, strides=2)

    d9 = Dense(df*16)(d8)
    d10 = LeakyReLU(alpha=0.2)(d9)
    validity = Dense(1, activation='sigmoid')(d10)

    return Model(d0, validity)

# Build and compile the discriminator
discriminator = build_discriminator(hr_shape, df)
discriminator.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

In [8]:
def build_generator(lr_shape=lr_shape, gf=gf, n_residual_blocks=n_residual_blocks, channels=channels):

    def residual_block(layer_input, filters):
        """
        Residual block described in paper
        """
        d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(layer_input)
        d = Activation('relu')(d)
        d = BatchNormalization(momentum=0.8)(d)
        d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
        d = BatchNormalization(momentum=0.8)(d)
        d = Add()([d, layer_input])
        return d

    def deconv2d(layer_input):
        """
        Layers used during upsampling
        """
        u = UpSampling2D(size=2)(layer_input)
        u = Conv2D(256, kernel_size=3, strides=1, padding='same')(u)
        u = Activation('relu')(u)
        return u

    # Low resolution image input
    img_lr = Input(shape=lr_shape)

    # Pre-residual block
    c1 = Conv2D(64, kernel_size=9, strides=1, padding='same')(img_lr)
    c1 = Activation('relu')(c1)

    # Propogate through residual blocks
    r = residual_block(c1, gf)
    for _ in range(n_residual_blocks - 1):
        r = residual_block(r, gf)

    # Post-residual block
    c2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(r)
    c2 = BatchNormalization(momentum=0.8)(c2)
    c2 = Add()([c2, c1])

    # Upsampling
    u1 = deconv2d(c2)
    u2 = deconv2d(u1)

    # Generate high resolution output
    gen_hr = Conv2D(channels, kernel_size=9, strides=1, padding='same', activation='tanh')(u2)

    return Model(img_lr, gen_hr)

# Build the generator
generator = build_generator(lr_shape, gf, n_residual_blocks, channels)


In [10]:

# High res. and low res. images
img_hr = Input(shape=hr_shape)
img_lr = Input(shape=lr_shape)

# Generate high res. version from low res.
fake_hr = generator(img_lr)

# Extract image features of the generated img
fake_features = vgg(fake_hr)

# For the combined model we will only train the generator
discriminator.trainable = False

# Discriminator determines validity of generated high res. images
validity = discriminator(fake_hr)

combined = Model([img_lr, img_hr], [validity, fake_features])
combined.compile(loss=['binary_crossentropy', 'mse'],
                      loss_weights=[1e-3, 1],
                      optimizer=optimizer)

        

def sample_images(epoch, dataset_name=dataset_name):
  """
  asdasd
  """

  imgs_hr, imgs_lr = load_data(batch_size=2, is_testing=True)
  fake_hr = generator.predict(imgs_lr)

  # Rescale images 0 - 1
  imgs_lr = (imgs_lr + 1)*511.5
  fake_hr = (fake_hr + 1)*511.5
  imgs_hr = (imgs_hr + 1)*511.5

  # Save generated images and the high resolution originals
  titles = ['Generated', 'Original']
  fig, axs = plt.subplots(1, 2)
  cnt = 0

  for col, image in enumerate([fake_hr, imgs_hr]):
    print(f'col: {col}')
    axs[col].imshow(image[0].astype(int))
    axs[col].set_title(titles[col])
    axs[col].axis('off')
    cnt += 1
  fig.savefig("images/%s/720/%d.png" % (dataset_name, epoch))
  plt.close()

  fig = plt.figure()
  plt.imshow(imgs_lr[0].astype(int))
  fig.savefig('images/%s/720/%d_lowres%d.png' % (dataset_name, epoch, 0))
  plt.close()



def train(epochs, generator=generator, discriminator=discriminator, disc_patch=disc_patch, vgg=vgg, combined=combined, batch_size=1, sample_interval=50):

    start_time = datetime.datetime.now()

    for epoch in range(epochs):

        # ----------------------
        #  Train Discriminator
        # ----------------------

        # Sample images and their conditioning counterparts
        imgs_hr, imgs_lr = load_data()


        # From low res. image generate high res. version

        fake_hr = generator.predict(imgs_lr)

        valid = np.ones((batch_size,) + disc_patch)
        fake = np.zeros((batch_size,) + disc_patch)

        # Train the discriminators (original images = real / generated = Fake)
        d_loss_real = discriminator.train_on_batch(imgs_hr, valid)
        d_loss_fake = discriminator.train_on_batch(fake_hr, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        #  Train Generator
        # Sample images and their conditioning counterparts
        imgs_hr, imgs_lr = load_data()

        # The generators want the discriminators to label the generated images as real
        valid = np.ones((batch_size,) + disc_patch)

        # Extract ground truth image features using pre-trained VGG19 model
        image_features = vgg.predict(imgs_hr)

        # Train the generators
        g_loss = combined.train_on_batch([imgs_lr, imgs_hr], [valid, image_features])
        # progress meter
        elapsed_time = datetime.datetime.now() - start_time
        print ("%d time: %s" % (epoch, elapsed_time))

        # save image at every other interval
        if epoch % sample_interval == 0:
            sample_images(epoch)


train(epochs=3500, batch_size=1, sample_interval=50) 


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 time: 0:00:19.631435


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


col: 0
col: 1


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

2032 time: 1:27:01.691503
2033 time: 1:27:04.235725
2034 time: 1:27:06.780401
2035 time: 1:27:09.326363
2036 time: 1:27:11.876520
2037 time: 1:27:14.441248
2038 time: 1:27:17.003301
2039 time: 1:27:19.573248
2040 time: 1:27:22.119268
2041 time: 1:27:24.685673
2042 time: 1:27:27.231830
2043 time: 1:27:29.783388
2044 time: 1:27:32.336022
2045 time: 1:27:34.898414
2046 time: 1:27:37.443176
2047 time: 1:27:39.996892
2048 time: 1:27:42.542153
2049 time: 1:27:45.092229
2050 time: 1:27:47.638983
col: 0
col: 1
2051 time: 1:27:50.864398
2052 time: 1:27:53.417688
2053 time: 1:27:55.970115
2054 time: 1:27:58.514546
2055 time: 1:28:01.057399
2056 time: 1:28:03.607159
2057 time: 1:28:06.158196
2058 time: 1:28:08.712676
2059 time: 1:28:11.243530
2060 time: 1:28:13.806895
2061 time: 1:28:16.355409
2062 time: 1:28:18.924974
2063 time: 1:28:21.464410
2064 time: 1:28:24.017026
2065 time: 1:28:26.579415
2066 time: 1:28:29.114517
2067 time: 1:28:31.650638
2068 time: 1:28:34.205474
2069 time: 1:28:36.75824

2344 time: 1:40:20.422162
2345 time: 1:40:22.974324
2346 time: 1:40:25.505247
2347 time: 1:40:28.045053
2348 time: 1:40:30.597782
2349 time: 1:40:33.160673
2350 time: 1:40:35.692695
col: 0
col: 1
2351 time: 1:40:38.943975
2352 time: 1:40:41.500142
2353 time: 1:40:44.047585
2354 time: 1:40:46.596822
2355 time: 1:40:49.136929
2356 time: 1:40:51.685582
2357 time: 1:40:54.230436
2358 time: 1:40:56.760277
2359 time: 1:40:59.294083
2360 time: 1:41:01.844337
2361 time: 1:41:04.392923
2362 time: 1:41:06.955156
2363 time: 1:41:09.503030
2364 time: 1:41:12.038808
2365 time: 1:41:14.574783
2366 time: 1:41:17.110732
2367 time: 1:41:19.652492
2368 time: 1:41:22.196458
2369 time: 1:41:24.730620
2370 time: 1:41:27.272762
2371 time: 1:41:29.814295
2372 time: 1:41:32.350947
2373 time: 1:41:34.890970
2374 time: 1:41:37.425788
2375 time: 1:41:39.971873
2376 time: 1:41:42.510745
2377 time: 1:41:45.070844
2378 time: 1:41:47.617631
2379 time: 1:41:50.143954
2380 time: 1:41:52.696569
2381 time: 1:41:55.23856

2656 time: 1:53:39.189086
2657 time: 1:53:41.744370
2658 time: 1:53:44.268418
2659 time: 1:53:46.806845
2660 time: 1:53:49.329902
2661 time: 1:53:51.869072
2662 time: 1:53:54.396141
2663 time: 1:53:56.945293
2664 time: 1:53:59.524251
2665 time: 1:54:02.074931
2666 time: 1:54:04.610303
2667 time: 1:54:07.143426
2668 time: 1:54:09.685373
2669 time: 1:54:12.240371
2670 time: 1:54:14.767103
2671 time: 1:54:17.319493
2672 time: 1:54:19.859890
2673 time: 1:54:22.397971
2674 time: 1:54:24.941146
2675 time: 1:54:27.487190
2676 time: 1:54:30.021258
2677 time: 1:54:32.556979
2678 time: 1:54:35.080851
2679 time: 1:54:37.646306
2680 time: 1:54:40.175287
2681 time: 1:54:42.707707
2682 time: 1:54:45.272055
2683 time: 1:54:47.804197
2684 time: 1:54:50.344667
2685 time: 1:54:52.904204
2686 time: 1:54:55.448124
2687 time: 1:54:57.995454
2688 time: 1:55:00.520424
2689 time: 1:55:03.059926
2690 time: 1:55:05.617143
2691 time: 1:55:08.160608
2692 time: 1:55:10.693640
2693 time: 1:55:13.226258
2694 time: 1

2968 time: 2:06:56.503553
2969 time: 2:06:59.036840
2970 time: 2:07:01.574573
2971 time: 2:07:04.116428
2972 time: 2:07:06.656823
2973 time: 2:07:09.196358
2974 time: 2:07:11.722104
2975 time: 2:07:14.269677
2976 time: 2:07:16.837095
2977 time: 2:07:19.391241
2978 time: 2:07:21.954318
2979 time: 2:07:24.490765
2980 time: 2:07:27.029368
2981 time: 2:07:29.560351
2982 time: 2:07:32.103622
2983 time: 2:07:34.639557
2984 time: 2:07:37.189485
2985 time: 2:07:39.735222
2986 time: 2:07:42.266555
2987 time: 2:07:44.800625
2988 time: 2:07:47.346261
2989 time: 2:07:49.907825
2990 time: 2:07:52.440364
2991 time: 2:07:54.992266
2992 time: 2:07:57.530838
2993 time: 2:08:00.070815
2994 time: 2:08:02.602787
2995 time: 2:08:05.131656
2996 time: 2:08:07.670474
2997 time: 2:08:10.210834
2998 time: 2:08:12.772515
2999 time: 2:08:15.306022
3000 time: 2:08:17.829948
col: 0
col: 1
3001 time: 2:08:21.076254
3002 time: 2:08:23.623932
3003 time: 2:08:26.156273
3004 time: 2:08:28.693010
3005 time: 2:08:31.25915

3280 time: 2:20:13.732802
3281 time: 2:20:16.269996
3282 time: 2:20:18.811124
3283 time: 2:20:21.348098
3284 time: 2:20:23.910082
3285 time: 2:20:26.436150
3286 time: 2:20:28.971637
3287 time: 2:20:31.525217
3288 time: 2:20:34.041558
3289 time: 2:20:36.569296
3290 time: 2:20:39.097350
3291 time: 2:20:41.626873
3292 time: 2:20:44.180593
3293 time: 2:20:46.722348
3294 time: 2:20:49.260830
3295 time: 2:20:51.802704
3296 time: 2:20:54.340495
3297 time: 2:20:56.874007
3298 time: 2:20:59.409903
3299 time: 2:21:01.930919
3300 time: 2:21:04.467201
col: 0
col: 1
3301 time: 2:21:07.685262
3302 time: 2:21:10.214250
3303 time: 2:21:12.729951
3304 time: 2:21:15.272900
3305 time: 2:21:17.801616
3306 time: 2:21:20.340713
3307 time: 2:21:22.869904
3308 time: 2:21:25.403978
3309 time: 2:21:27.933126
3310 time: 2:21:30.467480
3311 time: 2:21:32.998361
3312 time: 2:21:35.525770
3313 time: 2:21:38.050609
3314 time: 2:21:40.584476
3315 time: 2:21:43.111275
3316 time: 2:21:45.641559
3317 time: 2:21:48.17058